In [1]:
import numpy as np
import pandas as pd
import torch
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
#from pytorch_lightning.core.lightning import LightningModule
from torch.utils.data import DataLoader, Dataset
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
import re
import math
import random
import urllib.request
from torch.utils.data import DataLoader, Dataset
from transformers import PreTrainedTokenizerFast

C:\Users\USER\AppData\Local\Temp\ipykernel_17240\2468387676.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
c:\Users\USER\anaconda3\envs\KOGPT\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\USER\anaconda3\envs\KOGPT\lib\site-packages\torchaudio\backend\utils.py:62: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


# Get Data

In [2]:
Chatbot_Data = pd.read_csv("Test_V1.csv")
Chatbot_Data

,Q,A,label
0,"안녕하세요, 전주혁님! 귀하의 이력서를 읽어보았습니다.","안녕하세요, 제 이력서에 관심 가져주셔서 감사합니다. AI 엔지니어링과 데이터 과학...",0
1,"안녕하세요, 전주혁님, 귀하의 AI 경력에 대해 더 알고 싶습니다.",안녕하세요! AI와 데이터 과학에서의 제 경력을 자세히 설명해 드리겠습니다. 특히 ...,0
2,"안녕하세요, 전주혁님! 귀하의 프로젝트에 대해 듣고 싶습니다.","반갑습니다. 제가 참여한 다양한 프로젝트 중에서, 자율주행 센서의 안테나 성능 예측...",0
3,"안녕하세요, 전주혁님, 귀하의 데이터 과학 경험에 관심이 많습니다.",안녕하세요! 데이터 과학 분야에서의 제 경험에 대해 궁금하신 점이 있다면 언제든지 ...,0
4,"안녕하세요, 전주혁님! 귀하의 경력은 매우 인상적입니다.","안녕하세요, 저의 이력서가 인상적이라니 영광입니다. 특히, 다양한 AI 경진대회에서...",0
...,...,...,...
398,프로젝트 진행 중 기술적 어려움을 어떻게 극복하셨습니까?,"프로젝트 진행 중 기술적 어려움을 극복하기 위해, 저는 데이터 분석에 깊이 몰입했습...",2
399,"대회 준비 과정에서 가장 큰 리스크는 무엇이었고, 어떻게 대응하셨나요?","대회 준비 과정에서의 가장 큰 리스크는, 예측 모델의 정확도를 보장하는 것이었습니다...",2
400,"AI 대회의 정신적, 기술적 도전을 어떻게 극복하셨습니까?","AI 대회에서의 정신적, 기술적 도전을 극복하기 위해, 저는 체계적인 데이터 분석과...",2
401,AI 모델 구축 중 겪었던 가장 큰 기술적 어려움은 무엇이었나요?,AI 모델 구축 중 겪었던 가장 큰 기술적 어려움은 데이터의 복잡성과 예측 모델의 ...,2


In [3]:
Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = "</s>"
EOS = "</s>"
SENT = '<unused1>'
PAD = "<pad>"
MASK = "<unused0>"

# 허깅페이스 transformers 에 등록된 사전 학습된 koGTP2 토크나이저를 가져온다.
koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", bos_token=BOS, eos_token=EOS, unk_token="<unk>", pad_token=PAD, mask_token=MASK,)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [4]:
# 챗봇 데이터를 처리하는 클래스를 만든다.
class ChatbotDataset(Dataset):
    def __init__(self, chats, max_len=160):  # 데이터셋의 전처리를 해주는 부분
        self._data = chats
        self.max_len = max_len
        self.q_token = Q_TKN
        self.a_token = A_TKN
        self.sent_token = SENT
        self.eos = EOS
        self.mask = MASK
        self.tokenizer = koGPT2_TOKENIZER

    def __len__(self):  # chatbotdata 의 길이를 리턴한다.
        return len(self._data)

    def __getitem__(self, idx):  # 로드한 챗봇 데이터를 차례차례 DataLoader로 넘겨주는 메서드
        turn = self._data.iloc[idx]
        q = turn["Q"]  # 질문을 가져온다.
        q = re.sub(r"([?.!,])", r" ", q)  # 구둣점들을 제거한다.

        a = turn["A"]  # 답변을 가져온다.
        a = re.sub(r"([?.!,])", r" ", a)  # 구둣점들을 제거한다.

        q_toked = self.tokenizer.tokenize(self.q_token + q + self.sent_token)
        q_len = len(q_toked)

        a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)
        a_len = len(a_toked)

        #질문의 길이가 최대길이보다 크면
        if q_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        #질문의 길이 + 답변의 길이가 최대길이보다 크면
        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        # 답변 labels = [mask, mask, ...., mask, ..., <bos>,..답변.. <eos>, <pad>....]
        labels = [self.mask,] * q_len + a_toked[1:]

        # mask = 질문길이 0 + 답변길이 1 + 나머지 0
        mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)
        # 답변 labels을 index 로 만든다.
        labels_ids = self.tokenizer.convert_tokens_to_ids(labels)
        # 최대길이만큼 PADDING
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]

        # 질문 + 답변을 index 로 만든다.    
        token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
        # 최대길이만큼 PADDING
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]

        #질문+답변, 마스크, 답변
        return (token_ids, np.array(mask), labels_ids)


In [5]:
def collate_batch(batch):
    data = [item[0] for item in batch]
    mask = [item[1] for item in batch]
    label = [item[2] for item in batch]
    return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)


In [6]:
train_set = ChatbotDataset(Chatbot_Data, max_len=160)

#윈도우 환경에서 num_workers 는 무조건 0으로 지정, 리눅스에서는 2
train_dataloader = DataLoader(train_set, batch_size=32, num_workers=0, shuffle=True, collate_fn=collate_batch,)

In [7]:
print("start")
for batch_idx, samples in enumerate(train_dataloader):
    token_ids, mask, label = samples
    print("token_ids ====> ", token_ids)
    print("mask =====> ", mask)
    print("label =====> ", label)
print("end")

C:\Users\USER\AppData\Local\Temp\ipykernel_17240\4135955700.py:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:233.)
  return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)


start
token_ids ====>  tensor([[    2, 16020, 10293,  ...,     3,     3,     3],
        [    2,  9162, 11991,  ...,     3,     3,     3],
        [    2,  9138,  6831,  ...,     3,     3,     3],
        ...,
        [    2,  9504, 11052,  ...,     3,     3,     3],
        [    2, 16071, 42544,  ...,     3,     3,     3],
        [    2, 36294,  9552,  ...,     3,     3,     3]])
mask =====>  tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
label =====>  tensor([[9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3],
        ...,
        [9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3]])
token_ids ====>  tensor([[    2,  9504, 11052,  ...,     3,     3,     3],
        [    2, 11011,  8159,  ...,     3,     3,     3],
        [    2,  92

# Chatbot

In [8]:
Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'


In [9]:
koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
            bos_token=BOS, eos_token=EOS, unk_token='<unk>',
            pad_token=PAD, mask_token=MASK) 
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_set = ChatbotDataset(Chatbot_Data, max_len=160)
#윈도우 환경에서 num_workers 는 무조건 0으로 지정, 리눅스에서는 2
train_dataloader = DataLoader(train_set, batch_size=16, num_workers=0, shuffle=True, collate_fn=collate_batch,)
model.to(device)
model.train()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [11]:
learning_rate = 3e-5
criterion = torch.nn.CrossEntropyLoss(reduction="none")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epoch = 15
Sneg = -1e18

print ("start")
for epoch in range(epoch):
    for batch_idx, samples in enumerate(train_dataloader):
        optimizer.zero_grad()
        token_ids, mask, label = samples
        token_ids = token_ids.to('cuda')
        mask = mask.to('cuda')
        label = label.to('cuda')
        out = model(token_ids)
        out = out.logits      #Returns a new tensor with the logit of the elements of input
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
        mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))
        loss = criterion(mask_out.transpose(2, 1), label)
        # 평균 loss 만들기 avg_loss[0] / avg_loss[1] <- loss 정규화
        avg_loss = loss.sum() / mask.sum()
        avg_loss.backward()
        # 학습 끝
        optimizer.step()
    print(epoch, avg_loss)

print ("end")

start


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


0 tensor(16.2924, device='cuda:0', grad_fn=<DivBackward0>)
1 tensor(33.2540, device='cuda:0', grad_fn=<DivBackward0>)
2 tensor(47.4943, device='cuda:0', grad_fn=<DivBackward0>)
3 tensor(23.6305, device='cuda:0', grad_fn=<DivBackward0>)
4 tensor(24.8813, device='cuda:0', grad_fn=<DivBackward0>)
5 tensor(43.2311, device='cuda:0', grad_fn=<DivBackward0>)
6 tensor(11.1298, device='cuda:0', grad_fn=<DivBackward0>)
7 tensor(13.8311, device='cuda:0', grad_fn=<DivBackward0>)
8 tensor(25.9310, device='cuda:0', grad_fn=<DivBackward0>)
9 tensor(18.4143, device='cuda:0', grad_fn=<DivBackward0>)
10 tensor(8.5759, device='cuda:0', grad_fn=<DivBackward0>)
11 tensor(23.2061, device='cuda:0', grad_fn=<DivBackward0>)
12 tensor(17.5377, device='cuda:0', grad_fn=<DivBackward0>)
13 tensor(18.3230, device='cuda:0', grad_fn=<DivBackward0>)
14 tensor(13.2515, device='cuda:0', grad_fn=<DivBackward0>)
end


In [12]:
with torch.no_grad():
    while 1:
        q = input("user > ").strip()
        if q == "quit":
            break
        a = ""
        while 1:
            input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode(Q_TKN + q + SENT + A_TKN + a)).unsqueeze(dim=0)
            model = model.to('cpu')
            pred = model(input_ids)
            pred = pred.logits
            gen = koGPT2_TOKENIZER.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().numpy()
                                                         .tolist())[-1]
            if gen == EOS:
                break
            a += gen.replace("▁", " ")
        print("Chatbot > {}".format(a.strip()))

Chatbot > 반가워요  안녕하세요  전주혁입니다  저의 AI 프로젝트 경험에 대해 더 알고 싶으신가요
Chatbot > 프로젝트 소개하겠습니다  프로젝트 소개하겠습니다  프로젝트 소개하겠습니다  저는 전주혁이라고 합니다  주요 포인트는  LG AI Research와 LG 생활건강 주최의 AI 대회에서의 제품별 판매량 예측 프로젝트 참여입니다
Chatbot > 안녕하세요  저는 개발자 전주혁입니다  문제 해결을 위해 끝까지 노력하는 것이 제 특징이며  AI 분야에서 다양한 데이터를 다루는 데에 경험이 있습니다
